# Books : 


In [9]:
! pip install requests

! pip install beautifulsoup4

### Scrape the list of books From a certain page :

In [10]:
import requests
from bs4 import BeautifulSoup 

def get_book_list_page(page_url):
    response = requests.get(page_url)
    if response.status_code != 200 : 
        raise Exception('Failed to load page {}'.format(page_url))
    doc = BeautifulSoup(response.text , 'html.parser')
    return doc

-  Scrapping book list from different stores :

1. Culturel: 

In [11]:
culturel_url = 'https://www.culturel.tn/livre.html'
culturel_doc = get_book_list_page(culturel_url)

In [12]:
culturel_doc

<!DOCTYPE html>

<html lang="default">
<head>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type">
<title>Livre</title>
<meta content="Retrouvez les ouvrages disponibles à la vente dans notre librairie en ligne, librairies en Tunisie, Librairie en ligne, librairie culturel Menzah 6, librairie culturel ZAphir La Marsa, Librairie El Moez Menzah 1" name="description"/>
<meta content="librairie culturel, culturel librairie, vente livres tunisie, achat livre tunisie, librairie en ligne tunisie, librairie Tunisie,  librairie el moez el menzah,  librairie culturel Zaphir,  librairie culturel Menzah 6, librairie tunis, librairie la marsa, vente livre tunisie" name="keywords"/>
<meta content="index, follow" name="robots"/>
<meta content="TN-12" name="geo.region"/>
<meta content="Ariana" name="geo.placename"/>
<meta content="36.866537;10.164723" name="geo.position"/>
<meta content="36.866537, 10

2. Maison du Livre: 

In [13]:
ML_url = 'https://www.maisondulivretn.com/catalogue.php'
ML_doc = get_book_list_page(ML_url)

- Scrapping data from the culturel site: 

1. Getting the list of the links of the different links of the book pages :
    - the list of books is divided into different pages therefore to have access to the complete list we have to get the links  on each page 
    - Therefore, we need to access the list of the page numbers to access the links of each page at the exception of the first one

In [14]:
def get_book_list_links(doc):
    selection_class = 'pages'
    link_tags = doc.find_all('ul', {'class': selection_class})
    print(link_tags)
    # get the initial href prototype
    href_exp = link_tags[0].contents[3].contents[0]['href']
    #remove the page number : 
    href_exp = href_exp[:-1]
    num=1
    href = href_exp+str(num)
    req_stat = True
    books_pages_links = []
    while req_stat : 
        res = requests.get(href)
        if res.status_code == 200 :
            print(num)
            books_pages_links.append(href)
            num += 1
        else :
            req_stat = False

        # to limit the number of pages (200)
        if num == 201:
            req_stat = False
   
    # return link_tags
    return books_pages_links
   

In [15]:
book_list_pages = get_book_list_links(culturel_doc)

[<ul class="pages">
<li class="current"><a class="current">1</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=2">2</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=3">3</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=4">4</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=5">5</a></li>
<li>
<a class="next i-next" href="https://www.culturel.tn/livre.html?p=2" title="Suivant">
<i class="fa fa-chevron-circle-right"></i>
</a>
</li>
</ul>, <ul class="pages">
<li class="current"><a class="current">1</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=2">2</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=3">3</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=4">4</a></li>
<li><a href="https://www.culturel.tn/livre.html?p=5">5</a></li>
<li>
<a class="next i-next" href="https://www.culturel.tn/livre.html?p=2" title="Suivant">
<i class="fa fa-chevron-circle-right"></i>
</a>
</li>
</ul>]
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [359]:
book_list_details = []
def get_book_info_per_page(page_url):
    page_content = get_book_list_page(page_url)
    selection_class = 'col-md-12 prod-selec'
    book_content = page_content.find_all('div',{'class': selection_class})
    for book in book_content: 
        book_img_link_content = book.find('div' , {'class': 'img-prod'})
        # print('book_img_link_content', book_img_link_content.contents)
        book_page_link = book_img_link_content.find('a')['href']
        # print('book_page_link', book_page_link)
        book_img_link = book_img_link_content.find('a').find('img')['data-src']
        # print('book_img_link', book_img_link)
        book_total_des = book.find('div', {'class': 'allinfoallprod leftinfoallprod'}) # descr-prod
        # print('book_total_des', book_total_des)
        book_title = book_total_des.find('a').contents[0]
        # print('book_title', book_title)
        book_price_bloc = book.find('div',{'class': 'row listflexend'}).find('div',{'class': 'col-lg-3 col-sm-12'})
        # print('book_details', book_details)
        book_price = book_price_bloc.find('span',{'class': 'nv-prix'}).contents[0]
        # remove currency
        book_price = float(book_price[:-3].strip().replace(',' ,'.'))
        # print('book_price', book_price)
        book_description_bloc = book.find('div',{'class': 'row listflexend'}).find('div',{'class': 'col-lg-9 col-sm-12'})
        book_description = book_description_bloc.find('div',{'class': 'short_description'}).contents[0].strip()
        # print('book_description', book_description)
        book_auth_info = book_description_bloc.find('div',{'class': 'nameauteur'}).find_all('a')
        # print('book_auth_info', book_auth_info)
        book_author = book_auth_info[0].contents[0].strip()
        # print('book_author', book_author)
        book_editor = book_auth_info[1].contents[0].strip()
        # print('book_editor', book_editor)
        # get the book characteristics
        book_page_content = get_book_list_page(book_page_link) 
        book_carac = book_page_content.find('div', {'class':'product-collateral blocCaracteristiques'}).find_all('div', {'class':'col-md-6'})
        # print('len book_characteristics', len(book_carac))
        book_carac_p1 = book_carac[0].find_all('div', {'class':'informations-container'})
        # print('book_carac_p1', book_carac_p1)
        # print('test: ' , book_carac_p1[-1].find_all('div', {'class':'information'})[0].find('strong').contents[0]) #.contents[0].contents[0]
        if ( book_carac_p1[-1].find_all('div', {'class':'information'})[0].find('strong').contents[0] == 'Poids'):
             book_weight = book_carac_p1[-1].find_all('div', {'class':'information'})[1].contents[0].strip()[:-1]
        else:
            book_weight = '-'
        # book_weight = book_carac_p1[2].find_all('div', {'class':'information'})[1].contents[0].strip()[:-1]
        print('book_weight', book_weight)
        book_carac_p2 = book_carac[1].find_all('div', {'class':'informations-container'})
        # print('test: ')# ,book_carac_p2)#[0])#.find_all('div', {'class':'information'}))#[0].find('strong'))#.contents[0]) 
        if (book_carac_p2) and ( book_carac_p2[0].find_all('div', {'class':'information'})[0].find('strong').contents[0] == 'Format'):
             book_format = book_carac_p2[0].find_all('div', {'class':'information'})[1].contents[0].strip()
        else:
            book_format = '-'
        # book_format = book_carac_p2[0].find_all('div', {'class':'information'})[1].contents[0].strip()
        # print('book_format', book_format)
        # print("test: ", book_carac_p2[1].find_all('div', {'class':'information'})[0].contents[1].contents[0].contents[0])  
        if (book_carac_p2):
            print("test: ", len(book_carac_p2))
        else : 
            print('testing...')
        if (book_carac_p2) and (len(book_carac_p2)>1) and ( book_carac_p2[1].find_all('div', {'class':'information'})[0].contents[1].contents[0].contents[0] == 'Nb. de pages'):
            book_nb_pg = book_carac_p2[1].find_all('div', {'class':'information'})[1].contents[0].strip()[:-5]
        else:
            book_nb_pg = '-'
        print('book_nb_pg', book_nb_pg)
        # print("test: ", book_carac_p2[-1].find_all('div', {'class':'information'})[0].find('strong').contents[0])
        if (book_carac_p2) and ( book_carac_p2[-1].find_all('div', {'class':'information'})[0].find('strong').contents[0] == 'Dimensions'):
            book_dimensions_block = book_carac_p2[-1].find_all('div', {'class':'information'})[1].contents[0].strip().split("cm ×")
            #remove the last cm 
            book_dimensions_block[-1] =book_dimensions_block[-1][:-2]
            # print('book_dimensions_block', book_dimensions_block)
            book_dimensions = tuple(book_dimensions_block)
        else:
            book_dimensions_block = '-'
            # book_dimensions_block = book_carac_p2[-1].find_all('div', {'class':'information'})[1].contents[0].strip().split("cm ×")
           
        # print('book_dimensions', book_dimensions)
        book_dict = {
            'book_title': book_title,
            'book_img_link': book_img_link,
            'book_page_link': book_page_link,
            'book_price': book_price,
            'book_description': book_description,
            'book_author': book_author,
            'book_editor': book_editor,
            'book_weight': book_weight,
            'book_format': book_format,
            'book_nb_pg': book_nb_pg,
            'book_dimensions': book_dimensions
        }
        # print('book_dict:' , book_dict)
        book_list_details.append(book_dict)
        
        

def get_books_info(book_list_pages):
    for url in book_list_pages :
        get_book_info_per_page(url)
       
    
    return book_list_details


In [360]:
culturel_book_list_details = get_books_info(book_list_pages)

book_weight 224 
test:  3
book_nb_pg 96 
book_weight 134 
test:  3
book_nb_pg 32 
book_weight 110 
testing...
book_nb_pg -
book_weight 110 
testing...
book_nb_pg -
book_weight 278 
test:  3
book_nb_pg 550 
book_weight 220 
test:  3
book_nb_pg 415 
book_weight 354 
test:  3
book_nb_pg 544 
book_weight 118 
test:  3
book_nb_pg 29 
book_weight 374 
test:  3
book_nb_pg 576 
book_weight 110 
test:  1
book_nb_pg -
book_weight 224 
test:  3
book_nb_pg 96 
book_weight 134 
test:  3
book_nb_pg 32 
book_weight 110 
testing...
book_nb_pg -
book_weight 110 
testing...
book_nb_pg -
book_weight 278 
test:  3
book_nb_pg 550 
book_weight 220 
test:  3
book_nb_pg 415 
book_weight 354 
test:  3
book_nb_pg 544 
book_weight 118 
test:  3
book_nb_pg 29 
book_weight 374 
test:  3
book_nb_pg 576 
book_weight 110 
test:  1
book_nb_pg -
book_weight 224 
test:  3
book_nb_pg 96 
book_weight 134 
test:  3
book_nb_pg 32 
book_weight 110 
testing...
book_nb_pg -
book_weight 110 
testing...
book_nb_pg -
book_weight 

In [361]:
len(culturel_book_list_details)

2000

2. Transform data into a Dataframe and save it to a csv file 

In [363]:
import pandas as pd 
culturel_df = pd.DataFrame(culturel_book_list_details)
culturel_df.to_csv("culturel.csv", index=True)